In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('../inputs/ready_data/preprocessed_all_matches.csv')
df

,Division,Date,HomeTeam,AwayTeam,Full_Time_Result,Home_Team_Shots,Away_Team_Shots,Home_Team_Target_Shots,Away_Team_Target_Shots,Home_Team_Fouls_Commited,...,HOME_LOSSES_HOME,AWAY_WINS_HOME,AWAY_DRAWS_HOME,AWAY_LOSSES_HOME,HOME_WINS_AWAY,HOME_DRAWS_AWAY,HOME_LOSSES_AWAY,AWAY_WINS_AWAY,AWAY_DRAWS_AWAY,AWAY_LOSSES_AWAY
0,SP1,2006-08-26,Valencia,Betis,1,14.0,8.0,4.0,5.0,18.0,...,0,0,0,0,0,0,0,0,0,0
1,SP1,2006-08-27,Ath Bilbao,Sociedad,0,7.0,10.0,2.0,4.0,19.0,...,0,0,0,0,0,0,0,0,0,0
2,SP1,2006-08-27,Espanol,Gimnastic,2,20.0,18.0,6.0,7.0,25.0,...,0,0,0,0,0,0,0,0,0,0
3,SP1,2006-08-27,La Coruna,Zaragoza,1,14.0,13.0,6.0,6.0,22.0,...,0,0,0,0,0,0,0,0,0,0
4,SP1,2006-08-27,Osasuna,Getafe,2,15.0,7.0,7.0,2.0,26.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,SP1,2020-08-03,Valladolid,Ath Bilbao,2,19.0,10.0,2.0,5.0,9.0,...,2,7,3,3,3,4,7,1,7,5
5206,SP1,2020-08-03,Levante,Granada,0,11.0,10.0,3.0,7.0,21.0,...,3,8,2,3,3,0,11,3,2,8
5207,SP1,2020-08-03,Villarreal,Leganes,2,15.0,12.0,5.0,7.0,10.0,...,2,4,3,7,5,1,8,0,5,7
5208,SP1,2020-08-03,Betis,Real Madrid,1,14.0,13.0,3.0,4.0,8.0,...,4,9,4,0,1,6,6,7,4,2


# Añadir puntos y ranking

In [90]:
df = pd.read_csv('../inputs/ready_data/preprocessed_all_matches.csv')

home_points = []
away_points = []

home_win_streak = []
away_win_streak = []

team_data = {}
for _, row in df.iterrows():
    season = row.Season
    home_team = row.HomeTeam
    away_team = row.AwayTeam
    result = row.Full_Time_Result

    if season not in team_data:
        team_data[season] = {}
    
    if home_team not in team_data[season]:
        team_data[season][home_team] = {'Win_Streak': [], 'Points': []}

    if away_team not in team_data[season]:
        team_data[season][away_team] = {'Win_Streak': [], 'Points': []}
    
    home_points.append(sum(team_data[season][home_team]['Points']))
    away_points.append(sum(team_data[season][away_team]['Points']))

    home_win_streak.append(sum(team_data[season][home_team]['Win_Streak']))
    away_win_streak.append(sum(team_data[season][away_team]['Win_Streak']))

    if result == 0:
        team_data[season][home_team]['Win_Streak'] = []
        team_data[season][away_team]['Win_Streak'] = []

        team_data[season][home_team]['Points'].append(1)
        team_data[season][away_team]['Points'].append(1)

    elif result == 1:
        team_data[season][home_team]['Win_Streak'].append(1)
        team_data[season][away_team]['Win_Streak'] = []

        team_data[season][home_team]['Points'].append(3)
        team_data[season][away_team]['Points'].append(0)
        
    elif result == 2:
        team_data[season][home_team]['Win_Streak'] = []
        team_data[season][away_team]['Win_Streak'].append(1)

        team_data[season][home_team]['Points'].append(0)
        team_data[season][away_team]['Points'].append(3)

df['HOME_POINTS'] = home_points
df['AWAY_POINTS'] = away_points
df['HOME_WIN_STREAK'] = home_win_streak
df['AWAY_WIN_STREAK'] =  away_win_streak

df['POINT_DIFFERENCE'] = df['HOME_WIN_STREAK'] - df['AWAY_WIN_STREAK']
df['STREAK_DIFFERENCE'] = df['HOME_POINTS'] - df['AWAY_POINTS']


df['DIFFERENCE_TRUESKILL_MU_NO_RESET'] = df['HOME_TRUESKILL_MU_NO_RESET'] - df['AWAY_TRUESKILL_MU_NO_RESET']
df['DIFFERENCE_TRUESKILL_SIGMA_NO_RESET'] = df['HOME_TRUESKILL_SIGMA_NO_RESET'] - df['AWAY_TRUESKILL_SIGMA_NO_RESET']
df['DIFFERENCE_TRUESKILL_MU_SEASON'] = df['HOME_TRUESKILL_MU_SEASON'] - df['AWAY_TRUESKILL_MU_SEASON']
df['DIFFERENCE_TRUESKILL_SIGMA_SEASON'] = df['HOME_TRUESKILL_SIGMA_SEASON'] - df['AWAY_TRUESKILL_SIGMA_SEASON']


    

# Modelo básico

In [102]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, ShuffleSplit, learning_curve
from sklearn.model_selection import ShuffleSplit
import numpy as np
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
from yellowbrick.model_selection import learning_curve
%matplotlib inline




df_model = df[['Full_Time_Result','Home Attack Score', 'Home Middle Score',  'Home Budget', 'Away Budget',
        'Away Attack Score', 'Away Middle Score', 'Away Defensive Score',  'Difference_Overall_Score',
        'Difference_Attack_Score', 'Difference_Middle_Score', 'Difference_Defensive_Score', 'Difference_Budget',
        # 'HOME_ELO', 'AWAY_ELO',
        'DIFFERENCE_ELO',
        'DIFFERENCE_TRUESKILL_MU_NO_RESET', 'DIFFERENCE_TRUESKILL_SIGMA_NO_RESET', 
        'DIFFERENCE_TRUESKILL_MU_SEASON', 'DIFFERENCE_TRUESKILL_SIGMA_SEASON',
        'HOME_ID', 'AWAY_ID', 
        'HOME_WIN_STREAK', 'AWAY_WIN_STREAK',
        'POINT_DIFFERENCE', 'STREAK_DIFFERENCE'
        ]]

X = df_model.drop(['Full_Time_Result'], axis=1)
y = df_model.Full_Time_Result

X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state = 42)
# learning_curve(model, X, y, cv=5, scoring='accuracy')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy_score(y_test, y_pred)

0.5594781273983116

# Curva de train acc y test acc

In [82]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, ShuffleSplit, learning_curve
from sklearn.model_selection import ShuffleSplit
import numpy as np
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
from yellowbrick.model_selection import learning_curve
%matplotlib inline


df = pd.read_csv('../inputs/ready_data/preprocessed_all_matches.csv')
df = df[['Full_Time_Result', 'Home Overall Score', 'Home Attack Score', 'Home Middle Score', 'Home Defensive Score', 'Home Budget',
        'Away Overall Score', 'Away Attack Score', 'Away Middle Score', 'Away Defensive Score', 'Away Budget', 'Difference_Overall_Score',
        'Difference_Attack_Score', 'Difference_Middle_Score', 'Difference_Defensive_Score', 'Difference_Budget', 'HOME_ELO', 'AWAY_ELO', 'DIFFERENCE_ELO',
        'HOME_TRUESKILL_MU_NO_RESET', 'AWAY_TRUESKILL_MU_NO_RESET', 'HOME_TRUESKILL_SIGMA_NO_RESET', 'AWAY_TRUESKILL_SIGMA_NO_RESET', 'DRAW_CHANCE_NO_RESET',
        'HOME_TRUESKILL_MU_SEASON', 'AWAY_TRUESKILL_MU_SEASON', 'HOME_TRUESKILL_SIGMA_SEASON', 'AWAY_TRUESKILL_SIGMA_SEASON', 'DRAW_CHANCE_SEASON', 'HOME_ID', 'AWAY_ID',
        'HOME_WINS_HOME', 'HOME_DRAWS_HOME', 'HOME_LOSSES_HOME', 'AWAY_WINS_HOME', 'AWAY_DRAWS_HOME', 'AWAY_LOSSES_HOME', 'HOME_WINS_AWAY', 'HOME_DRAWS_AWAY', 
        'HOME_LOSSES_AWAY', 'AWAY_WINS_AWAY', 'AWAY_DRAWS_AWAY', 'AWAY_LOSSES_AWAY']]

X = df.drop(['Full_Time_Result'], axis=1)
y = df.Full_Time_Result

X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state = 42)

model = SVC()

learning_curve(model, X, y, cv=5, scoring='accuracy')
#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)




KeyboardInterrupt: 